In [1]:
import sys
import os

# Add the current directory to the Python path
sys.path.append(os.getcwd())

In [3]:
from dotenv import load_dotenv

load_dotenv("I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/.env")

api_key = os.getenv("OPENAI_API_KEY")

In [5]:
import os
import json
from langchain_core.prompts import ChatPromptTemplate  
from models import KnowledgeGraphLLM
# from My_Work.Code.Main_Implementation.models import KnowledgeGraphLLM

In [7]:
# load sample data
data = json.load(open('I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/outputs/sample/concept_abstracts_sample.json', 'r'))
relation_types = json.load(open('I:/11_DFKI_Hiwi/Work/01_Code/Graphusion/inputs/relation_types.json', 'r'))

In [9]:
concept_name = "question answering qa"
print('Sample concept:', concept_name)

# Joining abstracts for 1st concept with -- inbetween to make it a single string

abstracts = ' --  '.join(data[concept_name]['abstracts'])
print('Used abstracts:', abstracts)

Sample concept: question answering qa
Used abstracts: A crucial issue of current text generation models is that they often uncontrollably generate text that is factually inconsistent with inputs. Due to lack of annotated data, existing factual consistency metrics usually train evaluation models on synthetic texts or directly transfer from other related tasks, such as question answering (QA) and natural language inference (NLI).Bias in synthetic text or upstream tasks makes them perform poorly on text actually generated by language models, especially for general evaluation for various tasks. To alleviate this problem, we propose a weakly supervised framework named WeCheck that is directly trained on actual generated samples from language models with weakly annotated labels.WeCheck first utilizes a generative model to infer the factual labels of generated samples by aggregating weak labels from multiple resources.Next, we train a simple noise-aware classification model as the target metr

In [11]:
len(abstracts)

60305